<a href="https://colab.research.google.com/github/adipai/statistical-data-pruning-analysis/blob/main/breast_cancer_sampling_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pmlb

In [37]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pmlb import fetch_data
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

## Data preprocessing

In [4]:
# Generic data pre-processing

def preprocess_data_train(df):

    # Count missing values before handling missing data
    missing_before = df.isnull().sum().sum()
    print("Number of missing values before handling:", missing_before)

    # Handle missing data
    imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
    df[df.select_dtypes(include=['float64', 'int64']).columns] = imputer.fit_transform(df.select_dtypes(include=['float64', 'int64']))

    # Count missing values after handling missing data
    missing_after = df.isnull().sum().sum()
    print("Number of missing values after handling:", missing_after)

    # Normalize numeric columns
    scaler = StandardScaler()
    df[df.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(df.select_dtypes(include=['float64', 'int64']))

    return df, scaler, imputer

def preprocess_data_test(df, scaler, imputer):
    # Count missing values before handling missing data
    missing_before = df.isnull().sum().sum()
    print("Number of missing values before handling in test_dataset:", missing_before)

    # Handle missing data
    df[df.select_dtypes(include=['float64', 'int64']).columns] = imputer.transform(df.select_dtypes(include=['float64', 'int64']))

    # Count missing values after handling missing data
    missing_after = df.isnull().sum().sum()
    print("Number of missing values after handling in test_dataset:", missing_after)

    # Normalize numeric columns
    df[df.select_dtypes(include=['float64', 'int64']).columns] = scaler.transform(df.select_dtypes(include=['float64', 'int64']))

    return df

## Experiments

### Dataset 1: Breast cancer

In [5]:
breast_cancer = fetch_data('breast_cancer')
breast_cancer.describe()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,target
count,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,2.664336,1.073427,4.062937,1.073427,1.167832,2.048951,0.468531,2.772727,0.237762,0.297203
std,1.011818,0.986680,2.151187,1.935321,0.443052,0.738217,0.499883,1.099006,0.426459,0.457828
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,3.000000,0.000000,1.000000,2.000000,0.000000,2.000000,0.000000,0.000000
50%,3.000000,2.000000,4.000000,0.000000,1.000000,2.000000,0.000000,3.000000,0.000000,0.000000
75%,3.000000,2.000000,5.000000,1.000000,1.000000,3.000000,1.000000,3.000000,0.000000,1.000000
max,5.000000,2.000000,10.000000,6.000000,2.000000,3.000000,1.000000,5.000000,1.000000,1.000000


## Training and testing using ML models

In [6]:
# Generic function to test synthetic data using LR, SVM, DT

def evaluate_models(X_train, X_test, y_train, y_test):

    # Initialize classifiers
    classifiers = {
        "Logistic Regression": LogisticRegression(),
        "SVM": SVC(),
        "Decision Tree": DecisionTreeClassifier()
    }

    # Results dictionary to store evaluation metrics
    results = {}

    # Iterate over classifiers
    for name, clf in classifiers.items():
        # Fit classifier
        clf.fit(X_train, y_train)

        # Predictions
        y_pred = clf.predict(X_test)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # AUC-ROC
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:,1]
        else:
            y_prob = clf.decision_function(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Confusion Matrix": cm
        }

        # Plot AUC-ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} - AUC-ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{name}_auc_roc_curve.png', dpi=300)
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{name} - Confusion Matrix')
        plt.savefig(f'{name}_confusion_matrix.png', dpi=300)
        plt.close()

    return results


In [7]:

y = breast_cancer['target']
X = breast_cancer.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

preprocessed_X_train, scaler, imputer = preprocess_data_train(X_train)
preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

X_train, y_train = preprocessed_X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = preprocessed_X_test.to_numpy(), y_test.to_numpy()


Number of missing values before handling: 0
Number of missing values after handling: 0
Number of missing values before handling in test_dataset: 0
Number of missing values after handling in test_dataset: 0


In [8]:
results = evaluate_models(X_train, X_test, y_train, y_test)

In [9]:
print(results)

{'Logistic Regression': {'Accuracy': 0.7931034482758621, 'Precision': 0.8, 'Recall': 0.4444444444444444, 'F1 Score': 0.5714285714285714, 'ROC AUC': 0.7291666666666667, 'Confusion Matrix': array([[38,  2],
       [10,  8]])}, 'SVM': {'Accuracy': 0.7758620689655172, 'Precision': 0.7777777777777778, 'Recall': 0.3888888888888889, 'F1 Score': 0.5185185185185185, 'ROC AUC': 0.7833333333333333, 'Confusion Matrix': array([[38,  2],
       [11,  7]])}, 'Decision Tree': {'Accuracy': 0.6724137931034483, 'Precision': 0.4782608695652174, 'Recall': 0.6111111111111112, 'F1 Score': 0.5365853658536586, 'ROC AUC': 0.6749999999999999, 'Confusion Matrix': array([[28, 12],
       [ 7, 11]])}}


# Intelligent Pruning

In [35]:
def find_majority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    max_label = max(zip(counts, labels))[1]
    indices_with_max_label = np.where(y == max_label)[0]
    X_maj, y_maj = X[indices_with_max_label], y[indices_with_max_label]

    # Exclude majority class samples
    indices_without_max_label = np.where(y != max_label)[0]
    X_remaining, y_remaining = X[indices_without_max_label], y[indices_without_max_label]

    return X_maj, y_maj, X_remaining, y_remaining, min(counts)

def do_clustering(X, y, labels):
  clustered_X = defaultdict(list)
  clustered_y = defaultdict(list)

  for i, label in enumerate(labels):
      clustered_X[label].append(X[i])
      clustered_y[label].append(y[i])

  # Sort clustered_X and clustered_y in descending order based on the length of values in each dictionary
  sorted_clustered_X = dict(sorted(clustered_X.items(), key=lambda x: -len(x[1])))
  sorted_clustered_y = dict(sorted(clustered_y.items(), key=lambda x: -len(x[1])))

  return sorted_clustered_X, sorted_clustered_y


def intelligent_prune_data(pruning_samps, clustered_X, clustered_y, per_cluster_pruning_ratio=0.7, seed=42):

  random.seed(seed)
  pruning_ratios_X_maj, pruning_ratios_y_maj = defaultdict(list), defaultdict(list)
  for pruning_samp in pruning_samps:
    samps = 0
    # print("For Pruning samps: ", pruning_samp)
    prune_samps = pruning_samp
    clustered_X_new = defaultdict(list)
    clustered_y_new = defaultdict(list)
    # Iterate over the sorted dictionaries
    for label, values_X in clustered_X.items():
        # Calculate the number of samples to prune
        # print("#Samples left to prune: ", prune_samps)
        num_samples_to_prune = int(prune_samps * per_cluster_pruning_ratio)
        if(num_samples_to_prune > len(values_X)):
          num_samples_to_prune = len(values_X)//2
          prune_samps -= num_samples_to_prune
        else:
          prune_samps -= num_samples_to_prune
        # print("#Samples to prune in current cluster: ", num_samples_to_prune)

        # Randomly choose samples to prune
        indices_to_prune = random.sample(range(len(values_X)), num_samples_to_prune)

        # Prune the samples from clustered_X and clustered_y
        clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in indices_to_prune]
        clustered_y_new[label] = [clustered_y[label][i] for i in range(len(clustered_y[label])) if i not in indices_to_prune]
        # print("Size of clustered_X_new[label]: ", len(clustered_X_new[label]))
        # print("Size of clustered_y_new[label]: ", len(clustered_y_new[label]))

    while(prune_samps > 0):
        for label, values_X in clustered_X_new.items():
          if(prune_samps <=0):
            break

          index_to_prune = random.sample(range(len(values_X)), 1)
          clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in index_to_prune]
          clustered_y_new[label] = [clustered_y_new[label][i] for i in range(len(clustered_y_new[label])) if i not in index_to_prune]

          prune_samps -= 1

    for label in clustered_X_new:
        pruning_ratios_X_maj[pruning_samp].extend(clustered_X_new[label])
        pruning_ratios_y_maj[pruning_samp].extend(clustered_y_new[label])

  return pruning_ratios_X_maj, pruning_ratios_y_maj

def combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining):

  pruning_ratios_X, pruning_ratios_y = defaultdict(list), defaultdict(list)
  for pruning_ratio in pruning_ratios:
    pruning_ratios_X[pruning_ratio].extend(pruning_ratios_X_maj[pruning_ratio])
    pruning_ratios_X[pruning_ratio].extend(X_remaining)

    pruning_ratios_y[pruning_ratio].extend(pruning_ratios_y_maj[pruning_ratio])
    pruning_ratios_y[pruning_ratio].extend(y_remaining)

  return pruning_ratios_X, pruning_ratios_y

def do_intelligent_pruning(X, y, pruning_ratio=None):

  X_maj, y_maj, X_remaining, y_remaining, min_class_samples = find_majority_data(X, y)
  kmeans = KMeans(n_clusters=3, random_state = 42)
  kmeans.fit(X_maj)
  labels = kmeans.labels_
  clustered_X, clustered_y = do_clustering(X_maj, y_maj, labels)

  pruning_best = len(X_maj)-min_class_samples
  pruning_samps = [int(pruning_best * step) for step in np.arange(0, 1.1, 0.2)]
  pruning_ratios = [ratio for ratio in np.arange(0, 120, 20)]

  pruning_ratios_X_maj, pruning_ratios_y_maj = intelligent_prune_data(pruning_samps, clustered_X, clustered_y)

  pruning_ratios_X, pruning_ratios_y = combine_data(pruning_samps, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining)

  return pruning_ratios_X, pruning_ratios_y, pruning_samps, pruning_ratios


# Random Pruning

In [38]:
"""
inputs:
X: np.array
y: np.array
percentage: from 0% upto 100%, enter int value
"""
def random_prune_data(X, y, percentage, seed = 42):
  np.random.seed(seed)
  labels_count = {}
  labels = np.unique(y)
  for label in labels:
    labels_count[label] = np.count_nonzero(y == label)
  max_label = min_label = labels[0]
  for label in labels_count:
    if labels_count[label] > labels_count[max_label]:
      max_label = label
    if labels_count[label] < labels_count[min_label]:
      min_label = label

  # print("Max", max_label, labels_count[max_label])
  # print("Min", min_label, labels_count[min_label])

  prune_counts = {}
  prune_indexes = {}
  for label in labels_count:
    prune_counts[label] = labels_count[label] - labels_count[min_label]
    prune_indexes[label] = np.where(y == label)[0]

  prune_amount = int((percentage / 100) * sum(map(lambda x: x[1], prune_counts.items())))
  prune_it = {}

  while prune_amount > 0:
    for label in labels:
      if (len(prune_indexes[label]) - labels_count[min_label]) > 0 and prune_amount > 0:
        random_index = np.random.choice(len(prune_indexes[label]))
        random_item = prune_indexes[label][random_index]
        prune_indexes[label] = np.delete(prune_indexes[label], random_index)
        if prune_it.get(label, None) is None:
          prune_it[label] = np.array([])
        prune_it[label] = np.append(prune_it[label], [random_item])
        prune_amount -= 1



  formatted_indexes = np.array([])
  for label in prune_indexes:
    formatted_indexes = np.append(formatted_indexes, prune_indexes[label])
  formatted_indexes = np.sort(formatted_indexes)
  new_arr = np.array([np.int64(i) for i in formatted_indexes])

  return X[new_arr], y[new_arr]

# Calling Intelligent Pruning

In [36]:
pruning_ratios_X, pruning_ratios_y, pruning_samps, pruning_ratios = do_intelligent_pruning(X_train, y_train)
results_intelligent_pruning = dict()
for pruning_samp, pruning_ratio in zip(pruning_samps, pruning_ratios):
  intelligent_pruned_X_train = pruning_ratios_X[pruning_samp]
  intelligent_pruned_y_train = pruning_ratios_y[pruning_samp]
  results_intelligent_pruning[pruning_ratio] = evaluate_models(intelligent_pruned_X_train, X_test, intelligent_pruned_y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


#Calling Random Pruning

In [39]:
results_random_pruning = dict()
for pruning_ratio in pruning_ratios:
  random_pruned_X_train, random_pruned_y_train = random_prune_data(X_train, y_train, pruning_ratio)
  results_random_pruning[pruning_ratio] = evaluate_models(random_pruned_X_train, X_test, random_pruned_y_train, y_test)